In [ ]:
!pip install -q langdetect
#!gdown 1VkNpuudQnlj7g5uUCNPJ4MKxFdDdh7bZ

In [2]:
import spacy
import string
import warnings

import numpy as np
import pandas as pd

from pprint import pprint
from IPython.utils import io
from tqdm.notebook import tqdm
from gensim.models import Word2Vec 
from langdetect import DetectorFactory, detect
from IPython.core.display import HTML, display
from IPython.display import Image
from spacy.lang.en.stop_words import STOP_WORDS

warnings.filterwarnings('ignore')
tqdm.pandas()

/var/folders/21/vgybxt6529g5wrtxj6dvczdm0000gn/T/ipykernel_43160/3023876184.py:13: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display


In [9]:
DATA = pd.read_csv("metadata.csv").sample(10000) # taking only 100000 research papers
DATA.reset_index(inplace=True, drop=True)
print(DATA.columns)
DATA.head(2)

Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id'],
      dtype='object')


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,1r5qdcex,NaN,WHO,Monitoring of the Situation with the Coronavir...,NaN,NaN,NaN,unk,NaN,2021,"Ponomarev, Yuriy Makarov Andrei Borzykh Ksenia...",NaN,NaN,#6390,NaN,NaN,NaN,NaN,233753285.0
1,pwlo7b3o,NaN,Medline,"Assessing Provider-, Clinic-, and Structural-L...",10.1007/s10461-021-03297-7,NaN,33974167,unk,Intimate partner violence (IPV) relates to HIV...,2021-05-11,"Caplon, Amy; Alexander, Kamila A; Kershaw, Tra...",AIDS and behavior,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1007/s10461-021-03297-7; ht...,234351144.0


In [10]:
DATA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   cord_uid          10000 non-null  object 
 1   sha               3564 non-null   object 
 2   source_x          10000 non-null  object 
 3   title             9998 non-null   object 
 4   doi               6157 non-null   object 
 5   pmcid             3647 non-null   object 
 6   pubmed_id         4645 non-null   object 
 7   license           10000 non-null  object 
 8   abstract          7821 non-null   object 
 9   publish_time      9979 non-null   object 
 10  authors           9798 non-null   object 
 11  journal           9103 non-null   object 
 12  mag_id            0 non-null      float64
 13  who_covidence_id  4646 non-null   object 
 14  arxiv_id          156 non-null    object 
 15  pdf_json_files    3564 non-null   object 
 16  pmc_json_files    2971 non-null   object 

For each incoming query, **calculate semantic similarity** with all documents in the dataset and **pick top N publications** from the dataset.

In [11]:
df_covid = pd.DataFrame(columns=['paper_id', 'title','abstract', 'doi'])
df_covid['paper_id'] = DATA.sha
df_covid['title'] = DATA.title
df_covid['abstract'] = DATA.abstract
df_covid['doi'] = DATA.doi

df_covid.head()

,paper_id,title,abstract,doi
0,NaN,Monitoring of the Situation with the Coronavir...,NaN,NaN
1,NaN,"Assessing Provider-, Clinic-, and Structural-L...",Intimate partner violence (IPV) relates to HIV...,10.1007/s10461-021-03297-7
2,8c90218f153bc836445bea1d40ba86ab9291e822,Tocilizumab Treatment Effect on Iron Homeostas...,BACKGROUND: Tocilizumab has been proposed as a...,10.1159/000522307
3,NaN,Remote asthma consultations in primary care,The COVID-19 pandemic has accelerated the move...,10.12968/pnur.2021.32.1.10
4,NaN,"Race, research, and urban education","The need to advance knowledge about policy, le...",NaN


In [12]:
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   paper_id  3564 non-null   object
 1   title     9998 non-null   object
 2   abstract  7821 non-null   object
 3   doi       6157 non-null   object
dtypes: object(4)
memory usage: 312.6+ KB


In [13]:
df_covid.drop_duplicates(['abstract'], inplace=True)
df_covid.dropna(inplace=True)
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2999 entries, 2 to 9996
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   paper_id  2999 non-null   object
 1   title     2999 non-null   object
 2   abstract  2999 non-null   object
 3   doi       2999 non-null   object
dtypes: object(4)
memory usage: 117.1+ KB


In [16]:
# set seed
DetectorFactory.seed = 0

# hold label - language
languages = []

# loop through each text
for ii in tqdm(range(0,len(df_covid))):
    #print(ii)
    text = df_covid.iloc[ii]['abstract'].split(" ")
    
    lang = "en"
    try:
        if len(text) > 50:
            lang = detect(" ".join(text[:50]))
        elif len(text) > 0:
            lang = detect(" ".join(text[:len(text)]))
    except Exception as e:
        all_words = set(text)
        try:
            lang = detect(" ".join(all_words))
        
        except Exception as e:        
            lang = "unknown"
            pass
    
    # Appending to language label   
    languages.append(lang)

  0%|          | 0/2999 [00:00<?, ?it/s]

In [17]:
languages_dict = {}
for lang in set(languages):
    languages_dict[lang] = languages.count(lang)
    
print("Total: {}\n".format(len(languages)))
print(languages_dict)

Total: 2999

{'en': 2975, 'it': 1, 'de': 7, 'es': 6, 'fr': 8, 'nl': 2}


#### Since most of the publications are in English, we can safely drop the rest


In [18]:
df_covid['language'] = languages
df_covid = df_covid[df_covid['language'] == 'en'] 
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2975 entries, 2 to 9996
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   paper_id  2975 non-null   object
 1   title     2975 non-null   object
 2   abstract  2975 non-null   object
 3   doi       2975 non-null   object
 4   language  2975 non-null   object
dtypes: object(5)
memory usage: 139.5+ KB


In [19]:
df_covid = df_covid.drop(['language'], axis = 1) 
df_covid.head()

,paper_id,title,abstract,doi
2,8c90218f153bc836445bea1d40ba86ab9291e822,Tocilizumab Treatment Effect on Iron Homeostas...,BACKGROUND: Tocilizumab has been proposed as a...,10.1159/000522307
10,a6bb18c9c4e3bdedfbbd56e0fd2fa319b0fbd3ab,What Drives Water Utility Selection of Pricing...,Water pricing is a demand management strategy ...,10.1007/s11269-021-03018-8
12,bcd2a6c3add5665ab3208ff47c2312decee04229,The Influence of Post-Traumatic Growth on Coll...,Purpose: This study used a moderated mediation...,10.3389/fpsyg.2021.665973
24,9401f4fde129eb859595879fd4883a0e2a0c8b68,Hospital course and discharge criteria for chi...,BACKGROUND: For children hospitalized with bro...,10.1002/jhm.2318
25,d48edb44f059aee48ca0e126f95e006bc18968b9,Persistence of SARS-CoV-2 specific B- and T-ce...,Background Monitoring the adaptive immune resp...,10.1016/j.medj.2021.02.001


In [20]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

In [21]:
custom_stop_words = [
    'doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure', 
    'rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'medrxiv', 'license', 'fig', 'fig.', 
    'al.', 'Elsevier', 'PMC', 'CZI', 'www'
]

for w in custom_stop_words:
    if w not in stopwords:
        stopwords.append(w)

## cleaning abstracts

In [22]:
def pre_processor(sentence):
    mytokens = sentence.split(' ')
    mytokens = [word.lower() for word in mytokens if word not in stopwords and word not in punctuations]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

df_covid["processed_abstract"] = df_covid["abstract"].progress_apply(pre_processor)
df_covid.head()

  0%|          | 0/2975 [00:00<?, ?it/s]

,paper_id,title,abstract,doi,processed_abstract
2,8c90218f153bc836445bea1d40ba86ab9291e822,Tocilizumab Treatment Effect on Iron Homeostas...,BACKGROUND: Tocilizumab has been proposed as a...,10.1159/000522307,background: tocilizumab proposed effective tre...
10,a6bb18c9c4e3bdedfbbd56e0fd2fa319b0fbd3ab,What Drives Water Utility Selection of Pricing...,Water pricing is a demand management strategy ...,10.1007/s11269-021-03018-8,water pricing demand management strategy addre...
12,bcd2a6c3add5665ab3208ff47c2312decee04229,The Influence of Post-Traumatic Growth on Coll...,Purpose: This study used a moderated mediation...,10.3389/fpsyg.2021.665973,purpose: this study moderated mediation model ...
24,9401f4fde129eb859595879fd4883a0e2a0c8b68,Hospital course and discharge criteria for chi...,BACKGROUND: For children hospitalized with bro...,10.1002/jhm.2318,background: for children hospitalized bronchio...
25,d48edb44f059aee48ca0e126f95e006bc18968b9,Persistence of SARS-CoV-2 specific B- and T-ce...,Background Monitoring the adaptive immune resp...,10.1016/j.medj.2021.02.001,background monitoring adaptive immune response...


In [23]:
abstracts = df_covid['processed_abstract'].values

## Let us use Spacy for faster tokenization
nlp = spacy.load('en_core_web_sm', disable=["tagger", "ner"])  
nlp.add_pipe('sentencizer')

def tokenize_sentences(sentence):
    sentence_corpus = []
    doc = nlp(sentence)
    sentences = [sent.text.strip() for sent in doc.sents]
    for sent in sentences:
        processed_sent_list = sent.split(" ")
        sentence_corpus.append(processed_sent_list)
    return sentence_corpus

df_covid['tokenized_abstract'] = df_covid['processed_abstract'].progress_apply(lambda x: tokenize_sentences(x))

corpus_data = df_covid['tokenized_abstract'].to_list()
word2vec_corpus = [item for items in corpus_data for item in items]

  0%|          | 0/2975 [00:00<?, ?it/s]

#### Using Gensim's API to train Word2Vec

#### Parameters:
- **input** : Corpus data
- **min_count** : Ignores all words with total frequency lower than this.
- **size** : Dimensionality of the output word vectors.
- **workers** : Use these many worker threads to train the model (=faster training with multicore machines).
- **sg** : Training algorithm: 1 for skip-gram; otherwise CBOW.
- **negative** : If > 0, negative sampling will be used, the int for negative specifies how many "noise words".

In [29]:
model = Word2Vec(word2vec_corpus, min_count=3, vector_size= 100, workers=4, window =5, sg = 1, negative=5)

#### Let us calculate the centroid for each abstract using the vectors of all the words incorporating the abstract.

In [24]:
a = [0.0]*100
df_covid["centroid"] = [a]*df_covid.shape[0]

for index, row in df_covid.iterrows():
    abstract = row['processed_abstract']
    total_sim = 0
    words = abstract.split(" ")
    centroid = np.array([0.0]*100)
    for word in words:
        try:
            b = model[word]
        except:
            continue
        centroid = np.add(centroid, b)
    
    df_covid.at[index,'centroid'] = centroid.tolist()

#### Let us create our base function, which when given a query will retrieve and rank documents based on similarity.

In [25]:
def rank_docs(model, query, df_covid, num) :

    cosine_list = []
    
    a = []
    query = query.split(" ")
    for q in query:
        try:
            a.append(model[q])
        except:
            continue
    
    for index, row in df_covid.iterrows():
        centroid = row['centroid']
        total_sim = 0
        for a_i in a:
            cos_sim = np.dot(a_i, centroid)/(np.linalg.norm(a_i)*np.linalg.norm(centroid))
            total_sim += cos_sim
        cosine_list.append((row['title'], row['doi'], total_sim)) 
    
    
    cosine_list.sort(key=lambda x:x[2], reverse=True) ## in Descending order 
    
    papers_list = []
    for item in cosine_list[:num]:
        papers_list.append((item[0], item[1], item[2]))
    return papers_list

In [26]:
def query(query, top_matches=10):
    model_to_use = model
    df_covid_to_use = df_covid
    return rank_docs(model_to_use, query, df_covid_to_use, top_matches)

In [31]:
query('origin of corona virus')

[('Tocilizumab Treatment Effect on Iron Homeostasis in Severe COVID-19 Patients',
  '10.1159/000522307',
  0),
 ('What Drives Water Utility Selection of Pricing Methods? Evidence from California',
  '10.1007/s11269-021-03018-8',
  0),
 ('The Influence of Post-Traumatic Growth on College Students’ Creativity During the COVID-19 Pandemic: The Mediating Role of General Self-Efficacy and the Moderating Role of Deliberate Rumination',
  '10.3389/fpsyg.2021.665973',
  0),
 ('Hospital course and discharge criteria for children hospitalized with bronchiolitis',
  '10.1002/jhm.2318',
  0),
 ('Persistence of SARS-CoV-2 specific B- and T-cell responses in convalescent COVID-19 patients 6-8 months after the infection',
  '10.1016/j.medj.2021.02.001',
  0),
 ('Health Delivery and Quality Assurance Programs for Mice',
  '10.1016/b978-012369454-6/50065-0',
  0),
 ('Imperfect innate immune antagonism renders SARS-CoV-2 vulnerable towards IFN-γ and -λ',
  '10.1101/2020.10.15.340612',
  0),
 ('Exploring